<a href="https://colab.research.google.com/github/lro99/stock_sentiment/blob/main/SentimentTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import pandas as pd
# finBERT
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [2]:
# get list of news
news = yf.Search("spy", news_count=10).news
for i in news:
  print(i['title'])

Exchange-Traded Funds, Equity Futures Lower Pre-Bell Monday as Caution Deepens Amid Intensifying Tariff Escalations
Andor season 2 review: A Star Wars miracle, an ode to rebellion
Trump vs. Powell Just Triggered a Market Earthquake--And Tesla Is Caught in the Crossfire
50-Year-Old Investor Craves 8%+ Yield, Snubs Ford, Chases Durable Dividends – 'I Am Willing To Consider Anything' For Monthly Income
‘Andor’ Season 2 Thrills While Showing How Growth Is About Making Hard Decisions
Stock market today: Dow drops 950 points, S&P 500, Nasdaq tumble as Trump bashes Fed's Powell again
5 Quality ETFs Topping S&P 500 Past Month
Take the Zacks Approach to Beat the Markets: Pro-Dex, Brookdale Senior Living, Coca-Cola in Focus
Complete Solaria, Inc. Announces Rebrand as SunPower
Celebrity Photos April 2025: Diego Luna, Bowen Yang, Julia Fox


In [5]:
# S&P500 yahoo

ticker = yf.Ticker("^GSPC")
historical = ticker.history(period="5y")
historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-22 00:00:00-04:00,2787.889893,2815.100098,2775.949951,2799.310059,5084920000,0.0,0.0
2020-04-23 00:00:00-04:00,2810.419922,2844.899902,2794.260010,2797.800049,5773530000,0.0,0.0
2020-04-24 00:00:00-04:00,2812.639893,2842.709961,2791.760010,2836.739990,5382810000,0.0,0.0
2020-04-27 00:00:00-04:00,2854.649902,2887.719971,2852.889893,2878.479980,5203930000,0.0,0.0
2020-04-28 00:00:00-04:00,2909.959961,2921.149902,2860.709961,2863.389893,5691160000,0.0,0.0
...,...,...,...,...,...,...,...
2025-04-14 00:00:00-04:00,5441.959961,5459.459961,5358.020020,5405.970215,5031440000,0.0,0.0
2025-04-15 00:00:00-04:00,5411.990234,5450.410156,5386.439941,5396.629883,4317110000,0.0,0.0
2025-04-16 00:00:00-04:00,5335.750000,5367.240234,5220.790039,5275.700195,4607750000,0.0,0.0


In [14]:
twohun = historical['Open'].count()
twoavg = historical['Close'][twohun-200:].mean()
twoavg

np.float64(5749.824602050781)

In [19]:
# percent change col

historical['Change'] = (historical['Close'] - historical['Close'].shift(1))/historical['Close'].shift(1) * 100
historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Change
Date,,,,,,,,
2020-04-22 00:00:00-04:00,2787.889893,2815.100098,2775.949951,2799.310059,5084920000,0.0,0.0,NaN
2020-04-23 00:00:00-04:00,2810.419922,2844.899902,2794.260010,2797.800049,5773530000,0.0,0.0,-0.053942
2020-04-24 00:00:00-04:00,2812.639893,2842.709961,2791.760010,2836.739990,5382810000,0.0,0.0,1.391806
2020-04-27 00:00:00-04:00,2854.649902,2887.719971,2852.889893,2878.479980,5203930000,0.0,0.0,1.471407
2020-04-28 00:00:00-04:00,2909.959961,2921.149902,2860.709961,2863.389893,5691160000,0.0,0.0,-0.524238
...,...,...,...,...,...,...,...,...
2025-04-14 00:00:00-04:00,5441.959961,5459.459961,5358.020020,5405.970215,5031440000,0.0,0.0,0.794471
2025-04-15 00:00:00-04:00,5411.990234,5450.410156,5386.439941,5396.629883,4317110000,0.0,0.0,-0.172778
2025-04-16 00:00:00-04:00,5335.750000,5367.240234,5220.790039,5275.700195,4607750000,0.0,0.0,-2.240837


In [30]:
# finBERT model
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

headline = news[9]['title']
res = finbert(headline)
print(res)

Device set to use cpu


[{'label': 'neutral', 'score': 0.9373624920845032}]
